In [4]:
import numpy as np
from collections import defaultdict
import scipy.stats
import math


In [5]:
def bin2int(arr):
    val = 0
    for i in range(len(arr)):
        val += arr[i]<<i
    return val


In [6]:
with open("parameter.txt", "r") as config_file:
    n, l = map(int, config_file.readline().split())
    h = np.array(list(map(float, config_file.readline().split())))
    sigma = float(config_file.readline()) 


In [7]:
print("n: ",n,"\nl: ",l)
print("Weights: ", h)
print("Sigma: ",sigma)

n:  2 
l:  2
Weights:  [0.7 0.5]
Sigma:  0.25


In [8]:
with open("train.txt") as train_file:
    I = list(map(int, train_file.readline()))

In [9]:
I[:10]

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0]

In [10]:
m = n+l-1
prior_probabilities = defaultdict(float)
transition_probabilities = np.zeros((2**m, 2**m))
clusters = defaultdict(list)
cluster_means = defaultdict(float)
cluster_CoVar = defaultdict(float)

In [11]:
samples = []
X = []
prev_cluster_class = -1

for k in range(0,len(I)-n+1):
    if k != len(I) - n:
        cluster_class = bin2int(I[k:k+n+1])
    else:
        cluster_class = bin2int(I[k:k+n])
    # print(train_bits[k:k+n+1])
    X.append(np.array(I[k:k+n]).dot(h) + np.random.normal(loc=0.0, scale=sigma))
    samples.append(I[k:k+n])
    prior_probabilities[cluster_class] += 1
    if prev_cluster_class !=-1:
        transition_probabilities[cluster_class][prev_cluster_class] += 1
    if k>=n-1:
        curr_X = []
        for i in range(1,l+1):
            curr_X.append(X[-i])
        curr_X.reverse()
        clusters[cluster_class].append(curr_X)
    prev_cluster_class = cluster_class
    


In [12]:


transition_probabilities /= transition_probabilities.sum(axis=1, keepdims=True)

for i in range(2**m):
    clusters[i] = np.array(clusters[i])

for i in range(2**m):
    cluster_means[i] = np.mean(clusters[i],axis=0)
    cluster_CoVar[i] = np.cov(clusters[i].T)

for i in range(2**m):
    prior_probabilities[i] /= len(samples)


In [13]:
with open("test.txt") as test_file:
    I_Test = list(map(int, test_file.readline()))

test_samples = []
test_X = []
# len(train_bits)-n+1
for k in range(0,len(I_Test )-n+1):
    test_X.append(np.array(I_Test [k:k+n]).dot(h) + np.random.normal(loc=0.0, scale=sigma))
    if k>=n-1:
        curr_X = []
        for i in range(1,l+1):
            curr_X.append(test_X[-i])
        curr_X.reverse()
    test_samples.append(curr_X)

In [14]:
Table = []
x = test_samples[0]
temp = {}
for j in range(2**m):
    cost = np.linalg.norm(x-cluster_means[j])
    temp[j] = (cost, None)
Table.append(temp)

In [16]:
for i in range(1, len(test_samples)):
    x = test_samples[i]
    temp = {}
    for j in range(2 ** m):

        min_cost = float("inf")
        parent = None
        for k in range(2**m):
            cost = Table[i-1][k][0] + np.linalg.norm(x-cluster_means[k])
            if cost < min_cost:
                min_cost = cost
                parent = k
                temp[j] = (min_cost, parent)
    Table.append(temp)

In [17]:
last = Table[-1]
last_class = None
parent = None
min_cost = float("inf")
for j in range(2**m):
    cost = last[j][0]
    if cost < min_cost:
        min_cost = cost
        last_class = j
        parent = last[j][1]

In [18]:
decision = []
for i in range(len(Table)-1, 0, -1):
    dictionary = Table[i]
    parent = dictionary[last_class][1]
    decision.append(last_class)
    last_class = parent


In [19]:
last = Table[0]
last_class = None
min_cost = float("inf")
for j in range(2**m):
    cost = last[j][0]
    if cost < min_cost:
        min_cost = cost
        last_class = j
decision.append(last_class)


In [20]:
result = []
for i in list(reversed(decision)):
    result.append(np.binary_repr(i, width=l)[0])

print("\n\n\nOriginal bits\t : ", I_Test[l-1:])
predictions = list(map(int, result))
print("Predictions\t : ", predictions)

result = np.array(predictions).reshape(1, len(predictions))
test = np.array(I_Test[l-1:]).reshape(1, len(I_Test)-l+1)

diff = result-test
print("\n\nAccuracy : ",  100 - len(result[result!=test]) / len(predictions) * 100, "%")
with open('out2.txt', 'w') as f:
    for item in reversed(predictions):
        f.write("%s" % item)




Original bits	 :  [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1]
Predictions	 :  [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0]


Accuracy :  54.54545454545455 %


In [21]:
def multivariate_normal(x, means, cov_mat):
        d = len(x)
        co_eff = math.sqrt(math.pow(2 * math.pi, d) * math.fabs(np.linalg.det(cov_mat)))
        x_mu = np.matrix(x) - np.matrix(means)
        Power = -0.5 * x_mu * np.linalg.inv(cov_mat) * x_mu.transpose()
        Exp = math.exp(Power)
        result = Exp  / co_eff
        return result

In [28]:
Table = []
x = test_samples[0]
temp = {}
eps = 0.0000000000001
for j in range(2**m):
    cost = math.log(multivariate_normal(x, cluster_means[j], cluster_CoVar[j])+eps) + math.log(prior_probabilities[j]+eps)
    temp[j] = (cost, None)
Table.append(temp)

In [29]:
for i in range(1, len(test_samples)):
    x = test_samples[i]
    temp = {}
    for j in range(2 ** m):
        max_cost = -float("inf")
        parent = None
        for k in range(2**m):
            cost = Table[i-1][k][0] + math.log(multivariate_normal(x, cluster_means[j], cluster_CoVar[j])+eps) + math.log(transition_probabilities[j][k] +eps)
            if cost > max_cost:
                max_cost = cost
                parent = k
                temp[j] = (max_cost, parent)
    Table.append(temp)

In [30]:
last = Table[-1]
last_class = None
parent = None
max_cost = -float("inf")
for j in range(2**m):
    cost = last[j][0]
    if cost > max_cost:
        max_cost = cost
        last_class = j
        parent = last[j][1]

In [31]:
decision = []
for i in range(len(Table)-1, 0, -1):
    dictionary = Table[i]
    parent = dictionary[last_class][1]
    decision.append(last_class)
    last_class = parent


In [32]:
last = Table[0]
last_class = None
max_cost = -float("inf")
for j in range(2**m):
    cost = last[j][0]
    if cost > max_cost:
        max_cost = cost
        last_class = j
decision.append(last_class)


In [33]:
result = []
for i in list(reversed(decision)):
    result.append(np.binary_repr(i, width=l)[0])

print("\n\n\nOriginal bits\t : ", I_Test[l-1:])
predictions = list(map(int, result))
print("Predictions\t : ", predictions)

result = np.array(predictions).reshape(1, len(predictions))
test = np.array(I_Test[l-1:]).reshape(1, len(I_Test)-l+1)

diff = result-test
print("\n\nAccuracy : ",  100 - len(result[result!=test]) / len(predictions) * 100, "%")

with open('out1.txt', 'w') as f:
    for item in reversed(predictions):
        f.write("%s" % item)




Original bits	 :  [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1]
Predictions	 :  [0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1]


Accuracy :  74.74747474747474 %
